<a href="https://colab.research.google.com/github/Gabriela2685/Simulacion2/blob/main/cine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simpy

#Cine

Un cine tiene una taquilla que vende boletos para 6 películas. Cuando se agotan los boletos para una película, todas las personas que esperan comprar entradas para esa película deben retirarse de la fila.

In [ ]:
import collections
import random
import simpy as sp

In [ ]:
x0 = 42
tickets = 50  #por pelicula
sim_time = 120  #minutos

In [ ]:
def moviegoer(env, movie, num_tickets, theater):
    #El cliente intenta adquirir un numero de boletos para una funcion
    with theater.counter.request() as my_turn:
        #Esperar turno hasta que se agoten los boletos
        result = yield my_turn | theater.sold_out[movie]

        #Comprobar si el cliente puede ser atendido o si los boletos se han agotado
        if my_turn not in result:
            theater.num_renegers[movie] += 1
            return

        #Comprobar si quedan suficientes boletos
        if theater.available[movie] < num_tickets:
            #El cliente sale de la fila
            yield env.timeout(0.5)
            return

        #Compra de boletos
        theater.available[movie] -= num_tickets
        if theater.available[movie] < 2:
            #Todos los boletos se han agotado
            theater.sold_out[movie].succeed()
            theater.when_sold_out[movie] = env.now
            theater.available[movie] = 0
        yield env.timeout(1)


In [ ]:
def customer_arrivals(env, theater):
    #Crear nuevos clientes hasta el tiempo de simulacion
    while True:
        yield env.timeout(random.expovariate(1 / 0.5)) #La llegada de los clientes sigue unas dist. exponencial

        movie = random.choice(theater.movies)
        num_tickets = random.randint(1, 6)
        if theater.available[movie]:
            env.process(moviegoer(env, movie, num_tickets, theater))

Theater = collections.namedtuple('Theater', 'counter, movies, available, '
                                            'sold_out, when_sold_out, '
                                            'num_renegers')

In [ ]:
#Configuración de la simulacion
random.seed(x0)
env = sp.Environment()

In [ ]:
#Crear e entorno de simulacion
counter = sp.Resource(env, capacity=1)
movies = ['Terrifier 3','The Substance','Venom 2',
          'Robot Salvaje','Sonrie 2','Guason 2']
available = {movie: tickets for movie in movies}
sold_out = {movie: env.event() for movie in movies}
when_sold_out = {movie: None for movie in movies}
num_renegers = {movie: 0 for movie in movies}
theater = Theater(counter, movies, available, sold_out, when_sold_out,
                  num_renegers)

In [ ]:
#Comienza simulacion
env.process(customer_arrivals(env, theater))
env.run(until = sim_time)

In [ ]:
#Analisis de resultados
print('Simulacion de cine, venta de boletos \n \n')
for movie in movies:
    if theater.sold_out[movie]:
        print('La funcion "%s" se vendio en "%.1f" minutos' % (movie, theater.when_sold_out[movie]))
        print('Numero de personas que se fueron: %s' % theater.num_renegers[movie])

Simulacion de cinema, venta de boletos 
 

La funcion "Terrifier 3" se vendio en "69.0" minutos
Numero de personas que se fueron: 15
La funcion "The Substance" se vendio en "76.5" minutos
Numero de personas que se fueron: 11
La funcion "Venom 2" se vendio en "86.0" minutos
Numero de personas que se fueron: 10
La funcion "Robot Salvaje" se vendio en "89.0" minutos
Numero de personas que se fueron: 11
La funcion "Sonrie 2" se vendio en "65.0" minutos
Numero de personas que se fueron: 15
La funcion "Guason 2" se vendio en "75.5" minutos
Numero de personas que se fueron: 14
